In [ ]:
# pip install loguru

In [ ]:
# import sys

# repo_path = "/Workspace/Users/opolo.holtz@amaris.com/.bundle/marvelous-databricks-course-OpoloHOLTZ/dev/files/src"
# sys.path.append(repo_path)

# # Vérifier que le chemin est bien pris en compte
# print(sys.path)

In [ ]:
import os

from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from defaultccc.config import ProjectConfig
from defaultccc.serving.model_serving import ModelServing

# spark session
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

# Load project config
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
catalog_name = config.catalog_name
schema_name = config.schema_name

In [ ]:
# Initialize feature store manager
model_serving = ModelServing(
    model_name=f"{catalog_name}.{schema_name}.default_ccc_model_basic", endpoint_name="default_ccc-model-serving"
)

In [ ]:
# Deploy the model serving endpoint
model_serving.deploy_or_update_serving_endpoint()

In [ ]:
# Create a sample request body
required_columns = config.num_features + config.cat_features

# Sample 1000 records from the training set
test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").toPandas()

# Sample 100 records from the training set
sampled_records = test_set[required_columns].sample(n=100, replace=True).to_dict(orient="records")
dataframe_records = [[record] for record in sampled_records]

In [ ]:
dataframe_records[0]

In [ ]:
model_serving.call_endpoint(dataframe_records[0])

In [ ]:
model_serving.evaluate_serving_model(
    test_set[required_columns + [config.target]].sample(n=100, replace=True), config.target
)